# Open/Closed eyes detector using MediaPipe 

## Importing libraries

In [10]:
from pynq.lib.video import DisplayPort, VideoMode, PIXEL_RGB
import cv2, time, math
import mediapipe as mp

## Defining camera I/O

In [11]:
# --- I/O ------------------------------------------------------------------
Mode = VideoMode(640, 480, 24)
dp    = DisplayPort();  dp.configure(Mode, PIXEL_RGB)
cap   = cv2.VideoCapture(0 + cv2.CAP_V4L2)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

True

## Feature extraction definition and settings

In [12]:
# --- FaceMesh -------------------------------------------------------------
mp_face  = mp.solutions.face_mesh
face_mesh = mp_face.FaceMesh(static_image_mode=False,
                             max_num_faces=1,
                             refine_landmarks=True)  # iris points

# Indici dei 6 landmark usati per l’EAR (occhio sinistro)
LIND = [33, 160, 158, 133, 153, 144]   # (p1…p6)

def ear(pts):
    # pts = list of 6 (x,y) pixel tuples
    A = math.dist(pts[1], pts[5])
    B = math.dist(pts[2], pts[4])
    C = math.dist(pts[0], pts[3])
    return (A + B) / (2.0 * C)

THRESH = 0.25       # chiuso se EAR < THRESH
INTERVAL = 0.0001    # secondi tra due catture

## Exec loop

In [13]:
try:
    while True:
        t0 = time.time()
        # prova a leggere
        ret, frame = cap.read()
        # se non funziona la telecamera, interrompi
        if not ret:  break

        # FaceMesh => landmark
        rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        res = face_mesh.process(rgb)
        state = "UNKNOWN"
        if res.multi_face_landmarks:
            lm = res.multi_face_landmarks[0].landmark
            h, w = frame.shape[:2]
            pts = [(int(lm[i].x * w), int(lm[i].y * h)) for i in LIND]
            _ear = ear(pts)
            state = "CHIUSI" if _ear < THRESH else "APERTI"
            # disegna una linea sullo schermo
            cv2.putText(frame, f"OCCHI: {state}", (10,35),
                        cv2.FONT_HERSHEY_SIMPLEX, 1.0,(0,255,0),2)

        out = dp.newframe();  out[:] = frame;  dp.writeframe(out)
        # Stampa a console (opzionale)
        outstring = "Occhi:" + str(state)
        print(outstring, end="\r")

        # attende fino a INTERVAL s dall’inizio del ciclo
        time.sleep(max(0, INTERVAL - (time.time() - t0)))

except KeyboardInterrupt:
    cap.release() 
    dp.stop()
    print("\nServizio Interrotto")

W0000 00:00:1747498124.239270    6393 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1747498124.312693    6394 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


Occhi:UNKNOWN
Servizio Interrotto
